In [91]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool
import agentops

from pydantic import Field,BaseModel
from typing import List

from tavily import TavilyClient

import os

In [66]:
from dotenv import load_dotenv

load_dotenv()

# Get the key

os.environ["AGENTOPS_API_KEY"] = os.getenv("AGENTOPS_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["tvly_YOUR_API_KEY"] =os.getenv("tvly_YOUR_API_KEY")

agentops.init(
    api_key=os.getenv("AGENTOPS_API_KEY"),
    skip_auto_end_session=True
)

🖇 AgentOps: AgentOps has already been initialized. If you are trying to start a session, call agentops.start_session() instead.


In [70]:
print(os.getenv("tvly_YOUR_API_KEY")) 

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


In [71]:
output_dir = "./ai-agent-output"
os.makedirs(output_dir, exist_ok=True)

base_llm=LLM(model="groq/llama-3.3-70b-versatile",
    temperature=0)

tavily_client = TavilyClient(api_key=os.getenv("tvly_YOUR_API_KEY"))

In [53]:
no_keywords=10

# Agent A

In [87]:
class SuggestedSearchQueries(BaseModel):
    queries: List[str] = Field(..., title="Suggested search queries to be passed to the search engine",
                               min_items=1, max_items=no_keywords)


search_queries_recommendation_agent=Agent(
    role="Search Queries Recommendation Agent",
    goal="\n".join([
        "To generate optimized and relevant search queries for flight comparisons.",
        "To ensure queries cover various flight options, including different dates, airlines, and price ranges.",
        "To enhance the accuracy of search results by considering user preferences, budget constraints, and travel flexibility.",
        "To provide diverse queries that increase the likelihood of finding the best available flights."
    ]),
    backstory="""This agent is designed to enhance the flight search experience by generating optimized search queries. 
    It considers various factors such as travel dates, and user preferences to provide a diverse set of search queries. 
    By intelligently crafting these queries based on the given context, the agent ensures that the search engine retrieves the most relevant and competitive flight options.""",
    llm=base_llm,
    verbose=True,

)


search_queries_recommendation_task = Task(
    description="\n".join([
        "FlyFinder aims to retrieve all available flights from {departure} to {destination} on {date} at the best prices, following a value-for-price strategy.",
        "The company prioritizes purchasing from these websites: [www.Trip.com, www.Mytrip.com, www.aircairo.com,www.skyscanner.com].",
        "The objective is to gather all available flights across the internet for comparison in a later stage.",
        "Generate up to {no_keywords} optimized search queries.",
        "Ensure that search queries are in {language}.",
        "Queries must directly lead to flight booking or product pages, avoiding blogs or generic listing pages."
    ]),
    expected_output="A JSON object containing a list of suggested search queries.",
    output_json=SuggestedSearchQueries,
    output_file=os.path.join(output_dir, "step_1_suggested_search_queries.json"),
    agent=search_queries_recommendation_agent

)


# Agent B

In [ ]:
@tool
def search_engine_tool(query:str):
    """Useful for search-based queries. Use this to find current information about any query related pages using a search engine"""
    return tavily_client.search(query)

search_engine_agent = Agent(
    role= "Search Engine Agent",
    goal="To search for flights based on the suggested search query",
    backstory="The agent is designed to help in looking for flights by searching for flights based on the suggested search queries.",
    llm=base_llm,
    verbose=True,
    tools=[search_engine_tool]

)



search_engine_task = Task(


    
)

# Run the AI Crew

In [88]:
FlyFinder_crew = Crew(
    agents=[
        search_queries_recommendation_agent,
        
    ],
    tasks=[
        search_queries_recommendation_task,
       
    ],
    process=Process.sequential,
    # knowledge_sources=[company_context]
)

In [90]:
crew_results=FlyFinder_crew.kickoff(
    inputs={
       "departure":"القاهرة (CAI)",
       "destination":"المدينة المنورة (MED)",
        "date":"20/2/2025",
        "no_keywords":no_keywords,
        "language":"Arabic",
       

    }
)

# Agent: Search Queries Recommendation Agent
## Task: FlyFinder aims to retrieve all available flights from القاهرة (CAI) to المدينة المنورة (MED) on 20/2/2025 at the best prices, following a value-for-price strategy.
The company prioritizes purchasing from these websites: [www.Trip.com, www.Mytrip.com, www.aircairo.com,www.skyscanner.com].
The objective is to gather all available flights across the internet for comparison in a later stage.
Generate up to 10 optimized search queries.
Ensure that search queries are in Arabic.
Queries must directly lead to flight booking or product pages, avoiding blogs or generic listing pages.


# Agent: Search Queries Recommendation Agent
## Final Answer: 
{
  "queries": [
    "تذاكر طيران من القاهرة إلى المدينة المنورة في 20/2/2025 على www.Trip.com",
    "رحلات جوية من القاهرة إلى المدينة المنورة في 20/2/2025 على www.Mytrip.com",
    "حجز طيران من القاهرة إلى المدينة المنورة في 20/2/2025 على www.aircairo.com",
    "اسعار تذاكر طيران من القاهرة إلى ال